# test notebook title

In [2]:
print("test notebook code cell")

test notebook code cell


In [ ]:
test_var = 5

In [26]:
algo_name = "xgboost_automated"
algo_script_path = "/github/workspace/{}/src/xgboost_automated.py".format(algo_name)
algo_req_path = "/github/workspace/{}/requirements.txt".format(algo_name)

In [25]:
%%writefile $algo_script_path
import Algorithmia
import json
import os.path
import joblib
import xgboost
import pandas as pd

#I'm generated via a notebook and pushed via Github Actions!

client = Algorithmia.client()

def load_model_config(config_rel_path="../model_config.json"):
    """Loads the model manifest with the following structure
    {
      "model_filepath": uploaded model path on Algorithmia data collection
      "model_origin_repo": model development repository, with the Github CI workflow
      "model_origin_commithash": SHA of the commit, triggering the CI workflow
      "model_uploaded_utc": UTC timestamp of the automated model upload
    }
    """
    config = []
    config_path = "{}/{}".format(os.path.dirname(__file__), (config_rel_path))
    if os.path.exists(config_path):
        with open(config_path) as json_file:
            config = json.load(json_file)
    return config


def load_model(config):
    """Loads the model object from the file at model_filePath key in config dict"""
    model_path = config["model_filePath"]
    model_file = client.file(model_path).getFile().name
    loaded_xgb = joblib.load(model_file)
    return loaded_xgb


config = load_model_config()
xgb = load_model(config)
assert xgb.steps[0][0] == "vect"
assert xgb.steps[1][0] == "tfidf"
assert xgb.steps[2][0] == "model"
print("All assertions are okay, we have a perfectly uploaded model!")

# API calls will begin at the apply() method, with the request body passed as 'input'
# For more details, see algorithmia.com/developers/algorithm-development/languages
def apply(input):
    series_input = pd.Series([input])
    result = xgb.predict(series_input)
    return {
        "sentiment": result.tolist()[0], 
        "predicting_model": config["model_filepath"],
        "model_origin": "Commit SHA: {} from repo: {}".format(config["model_origin_commithash"], config["model_origin_repo"])
    }

Writing ../xgboost_automated/xgboost_automated.py


In [16]:
%%writefile $algo_req_path
# I'm generated via a notebook and pushed via Github Actions!
algorithmia>=1.0.0,<2.0
scikit-learn
pandas
numpy
joblib
xgboost

Writing ../xgboost_automated/requirements.txt


FileNotFoundError: [Errno 2] No such file or directory: '../xgboost_automated/requirements.txt'